# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import time
import string
import inspect
import numpy as np

import cvxpy as cp
import gurobipy

import copy, signal
import csv, datetime
import random
import itertools
from tqdm import tqdm

import numpy.random as random_numpy
from joblib import Parallel, delayed

import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
from scipy.stats import uniform, truncnorm

from birkhoff import birkhoff_von_neumann_decomposition

rng = random_numpy.default_rng(1234)
CORES = 5 ## Number of parallel threads to run

from birkhoff_edited import fast_decomposition

import warnings
warnings.filterwarnings("ignore")

# Global variables

In [ ]:
# Ideally, enter absolute path 
home_folder = "./"

# Helper functions and ranking utilities

In [ ]:
# Logistical/storing functions
exec(open('helper_funcs.py').read())

In [ ]:
exec(open('utils.py').read())

# Algorithms

    Dimensions of inputs
    # W: m x n
    # P: m x g
    # U: g x n
    # L: g x n
    # x: m x n

In [ ]:
exec(open('algorithms.py').read())

# Code to Run Simulation

The code below is adapted from the [Noisy-Fair-Subset-Selection](https://github.com/AnayMehrotra/Noisy-Fair-Subset-Selection) repository

In [ ]:
def gen_P_toy(m, rand_gen=None):
    rng = get_rng(rand_gen)
    
    P = [[0,0] for i in range(m)]
    
    mu1 = 0.6; std1 = 0.001 # 5
    cnt1 = 0; rand1 = truncnorm.rvs((0-mu1)/std1, (1-mu1)/std1, loc=mu1, scale=std1, size=m, random_state=rng.integers(10000))
    
    mu2 = 0.05; std2 = 0.001 # 5
    cnt2 = 0; rand2 = truncnorm.rvs((0-mu2)/std2, (1-mu2)/std2, loc=mu2, scale=std2, size=m, random_state=rng.integers(10000))
    
    rand_mix = rng.uniform(0,1,m)
    
    
    fraction_women = 0.40
    
    for i in range(m):
        if rand_mix[i] < (fraction_women - mu2) / (mu1 - mu2): 
            P[i] = [rand1[cnt1],0]
            cnt1 += 1
        else: 
            P[i] = [rand2[cnt2],0]
            cnt2 += 1
            
        P[i][1] = 1 - P[i][0]
        
    P = np.array(P)
    
    return P.T

In [ ]:
def plot_res(results_mean, results_std, utility_mean, utility_std, ITER = 100,\
             fairness_measure = 'Fairness measure', name_occ_list = '', name_occ_list2 = '',\
             exp = 'Synthetic data', m = 100, n = 50, g = 2, ylims=(0.6,1.0), save = True, useILP=False):
    num_of_alg = results_mean.shape[1]
    num_of_const_steps = results_mean.shape[0]

    algo_names = ['This work', 'SJ', 'CSV [Greedy]', 'MC', 'GAK [Det-Greedy]', 'Uncons']
    algo_colors = ['#2FA3EB', '#F2B93F', '#F06B56', '#4DF06D', '#604EE6', '#000000', '#804539', 'purple', 'black']
    color = {}

    # Plot: const vs fairness measure
    fig, ax = plt.subplots()
    for i in range(num_of_alg):
        x_axis = np.linspace(2, 1, num_of_const_steps)
        
        res = results_mean[:, i].T
        res_err = results_std[:, i].T / np.sqrt(ITER)
        
        plt.errorbar(x_axis, res, yerr=res_err, fmt=('--' if i == 2 else '-'),\
                     color=algo_colors[i], label=algo_names[i], linewidth=4, alpha=0.9)
        
    ax.invert_xaxis()
    plt.title(f'{exp}\n$(m,n,g)=$({m},{n},{g}),ITER={ITER},occ_lists=[{name_occ_list},{name_occ_list2}].', fontsize=15)
    #plt.ylim(ylims[0], ylims[1])
    plt.ylim(np.min(results_mean) - 0.02, np.max(results_mean) + 0.02)    
    ax.set_ylabel(f'(Less fair)\t\t\t{fairness_measure}\t\t\t(More fair)',fontsize=23)
    ax.set_xlabel('(Looser constraint)\tFairness const. ($\\alpha$)\t(Stricter constraint)',fontsize=23)
    legend = plt.legend(loc='best', shadow=False, fontsize=20)
    plt.tick_params(axis='both', which='major', labelsize=16)
    ax.tick_params(axis='both', which='major', labelsize=16)
    
    if save: pdf_savefig()
    else: plt.show()
        
    # Plot: const vs utility measure
    fig, ax = plt.subplots()
    for i in range(num_of_alg):
        x_axis = np.linspace(2, 1, num_of_const_steps)
        
        util = utility_mean[:, i].T
        util_err = utility_std[:, i].T / np.sqrt(ITER)
        
        plt.errorbar(x_axis, util, yerr=util_err,\
                     color=algo_colors[i], label=algo_names[i], linewidth=4, alpha=0.7)    
    ax.invert_xaxis()
    ax.set_ylabel(f'Utility',fontsize=23)
    plt.title(f'{exp}\n$(m,n,g)=$({m},{n},{g}),ITER={ITER},occ_lists=[{name_occ_list},{name_occ_list2}].', fontsize=15)
    ax.set_xlabel('(Looser constraint)\tFairness const. ($\\alpha$)\t(Stricter constraint)',fontsize=23)
    plt.ylim(np.min(utility_mean) - 0.02, np.max(utility_mean) + 0.02)    
    legend = plt.legend(loc='best', shadow=False, fontsize=20)
    plt.tick_params(axis='both', which='major', labelsize=16)
    ax.tick_params(axis='both', which='major', labelsize=16)
    
    if save: pdf_savefig()
    else: plt.show()
        
    # Plot: fairness vs utility measure
    fig, ax = plt.subplots()
    for i in range(num_of_alg):
        res = results_mean[:, i].T
        res_err = results_std[:, i].T / np.sqrt(ITER)
        
        util = utility_mean[:, i].T
        util_err = utility_std[:, i].T / np.sqrt(ITER)
        
        plt.errorbar(res, util, xerr=res_err, yerr=util_err,\
                     color=algo_colors[i], label=algo_names[i], linewidth=4, alpha=0.7)
    plt.ylim(np.min(utility_mean) - 0.02, np.max(utility_mean) + 0.02)    
    plt.xlim(np.min(results_mean) - 0.02, np.max(results_mean) + 0.02)    
    plt.title(f'{exp}\n$(m,n,g)=$({m},{n},{g}),ITER={ITER},occ_lists=[{name_occ_list},{name_occ_list2}].', fontsize=15)
    ax.set_ylabel(f'Utility',fontsize=23)
    ax.set_xlabel(f'(Less fair)'+'\t'*23+f'{fairness_measure}'+'\t'*23+'(More fair)',fontsize=23)
    legend = plt.legend(loc='best', shadow=False, fontsize=20)
    plt.tick_params(axis='both', which='major', labelsize=16)
    ax.tick_params(axis='both', which='major', labelsize=16)
        
    if save: pdf_savefig()
    else: plt.show()

In [ ]:
def run_syn_exp(ITERS=20, num_of_const_steps=5, rND_k=49, dist=None,\
                fairness_measure=compute_weighted_risk_diff, fairness_measure_name='Risk diff.',\
                m=100, n=50, g=2, verbose=False, useILP=False, bias_factor=0.5):
    
    num_of_alg = 6
    
    if dist is None: dist = np.ones(g)/g
    
    results_mean = np.zeros((num_of_const_steps, num_of_alg))
    results_std = np.zeros((num_of_const_steps, num_of_alg))
    
    utility_mean = np.zeros((num_of_const_steps, num_of_alg))
    utility_std = np.zeros((num_of_const_steps, num_of_alg))
    
    for ijk, gamma in enumerate(np.linspace(2, 1, num_of_const_steps)):

        # fix fairness constraints
        L = np.zeros((g,n))
        U = get_const_from_dist([0.5*gamma, 0.5*gamma], m, n, g)
        
        results_per_const = [[] for i in range(num_of_alg)]
        utility_per_const = [[] for i in range(num_of_alg)]
        
        for exp_run in tqdm(range(ITERS)):        
            cnt = 0
            while True:
                try:
                    P = gen_P_toy(m, rand_gen=rng) # Probabilities
                    PT = np.round(P) # Infer attributes
                    trueP = get_true_P(P, m, n, g) # Get true attributes
                    W = get_biased_util(P, m, n, g, bias_factor=bias_factor)

                    # Find fair ranking 
                    if useILP:
                        x_our = noisy_rank_ilp(W, P, L, U)
                    else:
                        x_our = noisy_rank_cvz_rounding(W, P, L, U, verbose = False)

                    x_greedy = greedy_fair_ranking(W, PT, L, U)    
                    x_LP, birkhoff = noisy_rank_basic_rounding(W, PT, L, U, getBirkhoff=True)
                    a, rankings = extractBirkhoff(birkhoff, n) # Compute Birkhoff decomposition

                    x_SS = subset_selection_algorithm(W[:, 0], P, L[:, -1], U[:, -1], n)
                    Lp = get_lower_const_from_dist_linkedIn_det_greedy([0.5*1, 0.5*1], m, n, g)
                    Up = get_upper_const_from_dist_linkedIn_det_greedy([0.5*1, 0.5*1], m, n, g)
                    x_det_greedy = linkedIn_det_greedy(W, PT, Lp, Up, n) # this assumes that W is a rank 1 metric
                    
                    x_uncons = greedy_fair_ranking(W, PT, np.zeros_like(L), np.ones_like(L)*2*n)
                    
                except:
                    cnt += 1
                    if cnt > 10: break
                    continue
                else:
                    break
            
            # Compute fairness measures
            if verbose: print('This work:')
            rd_our = fairness_measure(x_our, trueP, dist, m, n, g, k=rND_k, verbose=False, P=P)
            if verbose: print('Greedy algorithm:')
            rd_greedy = fairness_measure(x_greedy, trueP, dist, m, n, g, k=rND_k, verbose=False, P=P)
            rd_LP = 0
            for i, r in enumerate(rankings):
                rd_LP += a[i] * fairness_measure(r, trueP, dist, m, n, g, k=rND_k, verbose=False, P=P)
            if verbose: print('Subset selection:')
            rd_SS = fairness_measure(x_SS, trueP, dist, m, n, g, k=rND_k, verbose=False, P=P)
            rd_det_greedy = fairness_measure(x_det_greedy, trueP, dist, m, n, g, k=rND_k)
            
            rd_uncons = fairness_measure(x_uncons, trueP, dist, m, n, g, k=rND_k)
            
                
            # Print and store results 
            if verbose: print('$'*15, rd_our, rd_LP, rd_greedy)
            results_per_const[0].append(rd_our)
            results_per_const[1].append(rd_LP)
            results_per_const[2].append(rd_greedy)
            results_per_const[3].append(rd_SS)
            results_per_const[4].append(rd_det_greedy)
            results_per_const[5].append(rd_uncons)
            
            utility_per_const[0].append(get_utility(W, x_our))
            
            u_LP = 0
            for i, r in enumerate(rankings):
                u_LP += a[i] * get_utility(W, r)
            
            utility_per_const[1].append(u_LP)        
            utility_per_const[2].append(get_utility(W, x_greedy))
            utility_per_const[3].append(get_utility(W, x_SS))
            utility_per_const[4].append(get_utility(W, x_det_greedy))
            utility_per_const[5].append(get_utility(W, x_uncons))
        
        results_mean[ijk] = np.array([np.mean(results_per_const[i]) for i in range(num_of_alg)])
        results_std[ijk]  = np.array([np.std(results_per_const[i]) for i in range(num_of_alg)])
        
        utility_mean[ijk] = np.array([np.mean(utility_per_const[i]) for i in range(num_of_alg)])
        utility_std[ijk] = np.array([np.std(utility_per_const[i]) for i in range(num_of_alg)])
    
    plot_res(results_mean, results_std, utility_mean, utility_std, ITER = ITERS,\
             fairness_measure = fairness_measure, name_occ_list = 'NA', name_occ_list2 = 'NA',\
             exp = 'Synthetic data', m = m, n = n, g = g, save = True, useILP=useILP)
    
    return results_mean, results_std, utility_mean, utility_std

# Simulations

In [ ]:
# Results with weighted risk difference
results_mean, results_std, utility_mean, utility_std = run_syn_exp(ITERS=500,\
                                        num_of_const_steps=9, dist=np.ones(2)/2,\
                                        fairness_measure=compute_weighted_risk_diff,\
                                        rND_k=5, m=500, n=25, g=2, bias_factor=1)

In [ ]:
# Results with weighted selection lift 
results_mean, results_std, utility_mean, utility_std = run_syn_exp(ITERS=500,\
                                        num_of_const_steps=9, dist=np.ones(2)/2,\
                                        fairness_measure=compute_weighted_selec_lift,\
                                        rND_k=5, m=500, n=25, g=2, bias_factor=1)